# 1. Demonstrate Transfer Learning

**Summary**

- 

In [1]:
# Import libraries and ignore warnings
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
np.random.seed(2)

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

import tensorflow as tf
from keras import models
from keras import layers
from keras import optimizers
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau, EarlyStopping
from keras.preprocessing import image

from keras.applications import VGG16
import os
import numpy as np
from keras.preprocessing.image import ImageDataGenerator

import os

Using TensorFlow backend.


In [ ]:
# Load the convolutional base from imagenet
conv_base = VGG16(weights='imagenet',
                  include_top=False,
                  input_shape=(64,64,3))

In [ ]:
train_dir = 'images/training'
validation_dir = 'images/testing'

In [ ]:
datagen = ImageDataGenerator(rescale=1./255)  # Rescale images to 0-1
batch_size = 32  # Number of batches to read from default 32

In [ ]:
def extract_features(directory, sample_count):
    """
    extract the features when processing images through the convolutional base. The features will be fed to 
    classification neural network
    """
    # Create emtpy array
    features = np.zeros(shape=(sample_count, 2, 2, 512))
    # Here I added the shape 
    labels = np.zeros(shape=(sample_count,2))
    # define the generator that is going to grab 20 images at a time and loop infinitly.
    generator = datagen.flow_from_directory(
        directory,
        target_size=(64,64),
        batch_size=batch_size,
        class_mode='categorical'
        )
    # Usefull for mapping label names
    label_map = (generator.class_indices)
    i=0
    
    # Extract features
    for inputs_batch, labels_batch in generator:
        features_batch = conv_base.predict(inputs_batch)
        features[i * batch_size : (i + 1) * batch_size] = features_batch
        labels[i * batch_size : (i + 1) * batch_size] = labels_batch
        i += 1
        # Break because the generator will loop infinitly
        if i * batch_size >= sample_count:
            break
    return features, labels, label_map

In [ ]:
total_training = 0
for folder in os.listdir("images/testing"):
    total_training += len(os.listdir( os.path.join('images/testing',folder)))
    print(len(os.listdir( os.path.join('images/testing',folder))))
print(total_training)

In [ ]:
train_features, train_labels, label_map = extract_features(train_dir, 1000)
validation_features, validation_labels, label_map = extract_features(validation_dir, 200)

In [ ]:

train_features = np.reshape(train_features, (1000, 2 * 2 * 512))
validation_features = np.reshape(validation_features, (200, 2 * 2 * 512))

In [ ]:
label_map2 = label_map


In [ ]:
model = models.Sequential()

model.add(layers.Dense(256, activation='relu', input_dim= 2 * 2 * 512))
model.add(layers.Dropout(0.2))
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dropout(0.2))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(2, activation='softmax'))


model.compile(optimizer=optimizers.RMSprop(lr=2e-5),
             loss='binary_crossentropy',
             metrics=['acc'])

history = model.fit(train_features, train_labels,
                   epochs = 150,
                   batch_size = 32,
                   validation_data=(validation_features, validation_labels))

In [ ]:
import matplotlib.pyplot as plt

acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(acc) + 1)

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and Validation accuracy')
plt.legend()
plt.figure()

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and Validation loss')
plt.legend()
plt.plot()

In [ ]:
import matplotlib.pyplot as plt
from keras.preprocessing import image

In [ ]:
label_map2


In [ ]:
def return_class(numeric_id):
    for class_pred, id_class in label_map2.items():    # for name, age in dictionary.iteritems():  (for Python 2.x)
        if id_class == numeric_id:
            return class_pred

In [ ]:
def return_prediction(img_src,img_src_name):
    test_features, test_labels, label_map = extract_features(img_src, 1)
    test_features = np.reshape(test_features, (1, 2 * 2 * 512))
    class_id = return_class(model.predict_classes(test_features)[0])
    
    img = image.load_img(img_src_name, target_size=(64, 64))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x /= 255.

    plt.imshow(x[0])                           
    plt.axis('off')
    plt.title(class_id)
    plt.show()

In [ ]:
return_prediction('images/unique_test','images/unique_test/class/justin_smile.jpg')